### Setup libraries and parameters needed later

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import matplotlib.pyplot as plt
import numpy as np

# list of OCR-errors noticed in account numbers while exploring dataset 
corrections = {'ACClOOO':'ACC1000', 'ACClOOl':'ACC1001', 'ACClOO2':'ACC1002', 'ACClOO3':'ACC1003', 'ACClOO4':'ACC1004', 
               'ACClOO5':'ACC1005', 'ACClOO6':'ACC1006', 'ACClOO7':'ACC1007', 'ACClOO8':'ACC1008', 'ACClOO9':'ACC1009'}

# a dictionary of month <-> number associated
set = {1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 
       7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}

# threshold for z-score (anomaly detection), e.g., z-score > 2 or < -2
threshold = 2.0

### Collect data from target file 

In [ ]:
file = "banking_data_assignment.csv"
data = pd.read_csv(file)
#print(type(data))
#display(data)

### General exploration

In [ ]:
# exploring OCR-errors in features except Amount
print(data['Account Number'].unique())
print(data['Transaction Type'].unique())
print(data['Description'].unique())

"""
Insights gained have been written as comments inside triple quotes.
Account feature contains OCR-errors. These would need to be corrected.
Transaction types seem okay.
Descriptions seem okay.
Amount feature is dealt with later.
"""

### Correct OCR-errors in account numbers

In [ ]:
# the account numbers seem as OCR-errors assuming they should be alphanumeric of form 'ACC+digits'
# correcting the account numbers 'lOO' -> '100'

data['Account Number'] = data['Account Number'].map(corrections).fillna(data['Account Number'])
#display(data)

### Check type of transaction dates

In [ ]:
print(type(data['Transaction Date'][0]))

"""
Transaction dates are strings currently.
They will be converted to datetime format for sorting and easier assessment.
"""

### Convert dates to datetime format and then sort

In [ ]:
data['Transaction Date'] = pd.to_datetime(data['Transaction Date'])

# sort the database based on dates to help assessment 
data_sorted = data.sort_values(by = 'Transaction Date', ignore_index = True)

#display(data_sorted)

### Preliminary cleaning of Amount feature

In [ ]:
# clean $ sign and convert string values to int
def clean(val: str) -> int:
    if val[0] == '$':
        val = val[1:]
    return int(val.split('.')[0])

data_sorted['Amount'] = np.array(map(clean, data_sorted['Amount']))
data_sorted.to_csv('Sorted_data.csv')

### Function to visualize the monthly ins and outs based on description and check for discrepancy

In [ ]:
def visualize(data: pd.DataFrame, month: int) -> tuple[plt.figure, int]:  
    data_month = data.loc[(data['Transaction Date'].dt.month == month)]
    #display(data_month)
    
    # separate monthly total
    data_month_clipped = data_month[data_month['Account Number'] != 'SUBTOTAL']
    month_total_row = data_month[data_month['Account Number'] == 'SUBTOTAL']
    #display(data_month)
    #display(month_total_row)
    
    # given sum 
    month_total = month_total_row.iat[0, 3]
    # group monthly sum based on descriptor
    group_sum = data_month_clipped.groupby('Description')['Amount'].sum()
    group_sum = group_sum.to_dict()
    #print(jan_total)

    global set
    plt.figure(figsize=(12, 6))
    plt.bar(group_sum.keys(), group_sum.values())
    if sum(group_sum.values()) == month_total:
        plt.title(f'{set[month]} || Calculated: {sum(group_sum.values())} and Given: {month_total} || No Discrepancy')
    else:
        plt.title(f'{set[month]} || Calculated {sum(group_sum.values())} and Given: {month_total} || Discrepancy')
    
    return plt, month_total

### Visualize amount data for all months

In [ ]:
yearly_sum = 0
for i in range(1, 13):
    figure, total = visualize(data_sorted, i)
    #figure.ioff()
    yearly_sum += total
    #figure.show()

"""All monthly plots show no discrepancy between calculated sum and given sum as of now (***indicates no errors***).
However, the task descriptions hints that withdrawals need to be negative values, which I agree with (***errors are definitely there***).
Logic then dictates that the provided monthly sum values are wrong themselves as they have been calculated using faulty amount data.
Therefore, monthly/yearly total cannot be considered authentic and used for verification.
The transformation of amount data is implemented next."""


### Transform Amount sign based on transaction type

In [ ]:
# Withdrawal, ATM Withdrawal, Card payment, Direct Debit can only be negative
condition = (data_sorted['Transaction Type'] == 'Withdrawal') | (data_sorted['Transaction Type'] == 'ATM Withdrawal') \
#| (data_sorted['Transaction Type'] == 'Card Payment') | (data_sorted['Transaction Type'] == 'Direct Debit')

data_sorted.loc[condition, 'Amount'] = -1 * abs(data_sorted['Amount'])

# Perhaps Deposit can only be positive???
#condition = (data_sorted['Transaction Type'] == 'Deposit')
#data_sorted.loc[condition, 'Amount'] = abs(data_sorted['Amount'])

data_sorted.to_csv('Sorted_with_corrected_withdrawal_data.csv')
#display(data_sorted)


### Visualize amount data again for all months with corrected withdrawal

In [ ]:
# this analysis is optional (doesn't convey anything) as monthly/yearly sum cannot be used for verification

yearly_sum = 0
for i in range(1, 13):
    figure, total = visualize(data_sorted, i)
    #figure.ioff()
    yearly_sum += total
    #figure.show()

### Function to detect anomalies using z-score metric

In [ ]:
# anomaly detection among features using z-score
def anomaly(data_sorted: pd.DataFrame, feature: str) -> None:
    global threshold
    types = data_sorted[feature].dropna().unique()
    #print(types)
    data_clipped = data_sorted[[feature, 'Amount']].dropna()
    #display(data_clipped)
    for i in types:
        z_dataset = data_clipped[data_clipped[feature] == i]
        #print(z_dataset)
        arr = z_dataset['Amount']
        z_dataset['z-score'] = (arr - arr.mean()) / arr.std()
        #print(z_dataset)
        anomaly = z_dataset[abs(z_dataset['z-score']) > threshold]
        #print(anomaly)
        if len(anomaly) > 0:
            for j in anomaly['z-score']:
                result = anomaly[anomaly['z-score'] == j].index.values
                print(f'{feature} -> {i}: Possible anomaly at index {result}')
        else:
            print(f'{feature} -> {i}: No anomaly')

### Detect anomalies in features

In [ ]:
# more features can be added to the list below to check for anomalies based on z-score
features = ['Transaction Type', 'Description', 'Account Number']
for i in features:
    anomaly(data_sorted, i)

"""
Some anomalies have been pointed out by the z-score metric threshold.
These can be looked at further.
"""